# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   index                  15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12466 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  mau                    15000 non-null  bool   
 15  re

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [4]:
import seaborn as sns
import patsy
from seaborn import load_dataset

import matplotlib.pyplot as plt

import numpy as np

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

### Item 01.

In [6]:
# Exibe os nomes das colunas no DataFrame df
df.columns

Index(['Unnamed: 0', 'data_ref', 'index', 'sexo', 'posse_de_veiculo',
       'posse_de_imovel', 'qtd_filhos', 'tipo_renda', 'educacao',
       'estado_civil', 'tipo_residencia', 'idade', 'tempo_emprego',
       'qt_pessoas_residencia', 'mau', 'renda'],
      dtype='object')

In [7]:
# Seleciona as features (X) e a variável alvo (y) do DataFrame df
X = df[['sexo', 'posse_de_veiculo','posse_de_imovel', 'qtd_filhos', 'tipo_renda', 'educacao',
       'estado_civil', 'tipo_residencia', 'idade', 'tempo_emprego', 'qt_pessoas_residencia']]

y = df['renda']

# Aplica one-hot encoding nas features categóricas
X = pd.get_dummies(X)

# Substitui valores booleanos por 1 e 0
X['posse_de_veiculo'] = X['posse_de_veiculo'].replace({True: 1, False: 0})
X['posse_de_imovel'] = X['posse_de_imovel'].replace({True: 1, False: 0})

# Exibe os nomes das colunas após as transformações
X.columns

Index(['posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos', 'idade',
       'tempo_emprego', 'qt_pessoas_residencia', 'sexo_F', 'sexo_M',
       'tipo_renda_Assalariado', 'tipo_renda_Bolsista',
       'tipo_renda_Empresário', 'tipo_renda_Pensionista',
       'tipo_renda_Servidor público', 'educacao_Primário',
       'educacao_Pós graduação', 'educacao_Secundário',
       'educacao_Superior completo', 'educacao_Superior incompleto',
       'estado_civil_Casado', 'estado_civil_Separado', 'estado_civil_Solteiro',
       'estado_civil_União', 'estado_civil_Viúvo', 'tipo_residencia_Aluguel',
       'tipo_residencia_Casa', 'tipo_residencia_Com os pais',
       'tipo_residencia_Comunitário', 'tipo_residencia_Estúdio',
       'tipo_residencia_Governamental'],
      dtype='object')

In [8]:
# Lista de colunas após as transformações e one-hot encoding
x_columns = ['posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos', 'idade',
       'tempo_emprego', 'qt_pessoas_residencia', 'sexo_F', 'sexo_M',
       'tipo_renda_Assalariado', 'tipo_renda_Bolsista',
       'tipo_renda_Empresário', 'tipo_renda_Pensionista',
       'tipo_renda_Servidor público', 'educacao_Primário',
       'educacao_Pós graduação', 'educacao_Secundário',
       'educacao_Superior completo', 'educacao_Superior incompleto',
       'estado_civil_Casado', 'estado_civil_Separado', 'estado_civil_Solteiro',
       'estado_civil_União', 'estado_civil_Viúvo', 'tipo_residencia_Aluguel',
       'tipo_residencia_Casa', 'tipo_residencia_Com os pais',
       'tipo_residencia_Comunitário', 'tipo_residencia_Estúdio',
       'tipo_residencia_Governamental']

In [9]:
# Renomeando as colunas substituindo espaços por underscores
renamed_columns = []
for name in x_columns:
    renamed_columns.append(name.replace(" ", "_"))

# Atribuindo os novos nomes às colunas
X.columns = renamed_columns

# Criando a fórmula para o modelo de regressão linear
formula = "renda ~ " + " + ".join(renamed_columns)
formula

'renda ~ posse_de_veiculo + posse_de_imovel + qtd_filhos + idade + tempo_emprego + qt_pessoas_residencia + sexo_F + sexo_M + tipo_renda_Assalariado + tipo_renda_Bolsista + tipo_renda_Empresário + tipo_renda_Pensionista + tipo_renda_Servidor_público + educacao_Primário + educacao_Pós_graduação + educacao_Secundário + educacao_Superior_completo + educacao_Superior_incompleto + estado_civil_Casado + estado_civil_Separado + estado_civil_Solteiro + estado_civil_União + estado_civil_Viúvo + tipo_residencia_Aluguel + tipo_residencia_Casa + tipo_residencia_Com_os_pais + tipo_residencia_Comunitário + tipo_residencia_Estúdio + tipo_residencia_Governamental'

In [10]:
# Preenchendo valores ausentes na coluna 'tempo_emprego' com zero
X['tempo_emprego'] = X['tempo_emprego'].fillna(0)

# Verificando se ainda existem valores ausentes no DataFrame X
X.isna().sum()

posse_de_veiculo                 0
posse_de_imovel                  0
qtd_filhos                       0
idade                            0
tempo_emprego                    0
qt_pessoas_residencia            0
sexo_F                           0
sexo_M                           0
tipo_renda_Assalariado           0
tipo_renda_Bolsista              0
tipo_renda_Empresário            0
tipo_renda_Pensionista           0
tipo_renda_Servidor_público      0
educacao_Primário                0
educacao_Pós_graduação           0
educacao_Secundário              0
educacao_Superior_completo       0
educacao_Superior_incompleto     0
estado_civil_Casado              0
estado_civil_Separado            0
estado_civil_Solteiro            0
estado_civil_União               0
estado_civil_Viúvo               0
tipo_residencia_Aluguel          0
tipo_residencia_Casa             0
tipo_residencia_Com_os_pais      0
tipo_residencia_Comunitário      0
tipo_residencia_Estúdio          0
tipo_residencia_Gove

In [11]:
# Dividindo o conjunto de dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Exibindo informações sobre o conjunto de treinamento
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11250 entries, 8222 to 3003
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   posse_de_veiculo               11250 non-null  int64  
 1   posse_de_imovel                11250 non-null  int64  
 2   qtd_filhos                     11250 non-null  int64  
 3   idade                          11250 non-null  int64  
 4   tempo_emprego                  11250 non-null  float64
 5   qt_pessoas_residencia          11250 non-null  float64
 6   sexo_F                         11250 non-null  uint8  
 7   sexo_M                         11250 non-null  uint8  
 8   tipo_renda_Assalariado         11250 non-null  uint8  
 9   tipo_renda_Bolsista            11250 non-null  uint8  
 10  tipo_renda_Empresário          11250 non-null  uint8  
 11  tipo_renda_Pensionista         11250 non-null  uint8  
 12  tipo_renda_Servidor_público    11250 non-nul

In [12]:
# Unindo os conjuntos de treinamento de features (X_train) e alvo (y_train) em um novo DataFrame (df_test)
df_test = X_train.merge(y_train, left_index=True, right_index=True)

### Ridge (Reduzindo o overfitting)

In [13]:
# Definindo um modelo de regressão linear usando a fórmula especificada e o conjunto de dados de treinamento
modelo = smf.ols(formula, data=df_test)

# Definindo uma lista de valores alpha para regularização
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# Listas para armazenar os valores de R² e AIC para diferentes valores de alpha
r2s = []
#aics = []

# Loop para ajustar o modelo regularizado com diferentes valores de alpha
for alpha in alphas:
    reg = modelo.fit_regularized(method='elastic_net',
                                refit=True,
                                L1_wt=0.01,
                                alpha=alpha)
    # Fazendo previsões no conjunto de teste
    y_pred = reg.predict(X_test)
    
    # Calculando e armazenando os valores de R²
    r2s.append(r2_score(y_test, y_pred))
    
    # AIC pode ser obtido utilizando reg.aic, mas está comentado neste código

# Imprimindo os resultados para cada valor de alpha
for i in range(len(alphas)):
    print(f"Para o alpha={alphas[i]}, o R² é {r2s[i]}")

Para o alpha=0, o R² é 0.23288443929195957
Para o alpha=0.001, o R² é 0.23288443929195957
Para o alpha=0.005, o R² é 0.23288443929195957
Para o alpha=0.01, o R² é 0.23288443929195957
Para o alpha=0.05, o R² é 0.23288443929195957
Para o alpha=0.1, o R² é 0.23288443929195957


R: O R² para todos os alfas é invariável.

### Lasso (reduzindo o overfitting e a multicolinearidade)

In [14]:
# Lista de valores alpha para regularização
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# Listas para armazenar os valores de R² e AIC para diferentes valores de alpha
r2s = []
#aics = []

# Loop para ajustar o modelo regularizado com diferentes valores de alpha
for alpha in alphas:
    # Ajustando o modelo usando elastic net com L1_wt=1 para regularização L1 (Lasso)
    reg = modelo.fit_regularized(method='elastic_net',
                                refit=True,
                                L1_wt=1,  # L1_wt=1 indica Lasso, L1_wt=0 indica Ridge
                                alpha=alpha)
    # Fazendo previsões no conjunto de teste
    y_pred = reg.predict(X_test)
    
    # Calculando e armazenando os valores de R²
    r2s.append(r2_score(y_test, y_pred))
    
    # AIC pode ser obtido utilizando reg.aic, mas está comentado neste código

# Imprimindo os resultados para cada valor de alpha
for i in range(len(alphas)):
    print(f"Para o alpha={alphas[i]}, o R² é {r2s[i]}")

Para o alpha=0, o R² é 0.23288443929195957
Para o alpha=0.001, o R² é 0.23288443929195957
Para o alpha=0.005, o R² é 0.23288443929195957
Para o alpha=0.01, o R² é 0.23289887334456827
Para o alpha=0.05, o R² é 0.23289887334456827
Para o alpha=0.1, o R² é 0.23289887334456816


### Stepwise (selecionando as variáveis mais relevantes para um modelo)

In [15]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out=0.05, 
                       verbose=True):
    """ Realiza uma seleção de características avante-retrocedente
    baseada no valor de p do OLS (Ordinary Least Squares) do statsmodels.api.
    Argumentos:
        X - pandas.DataFrame com as características candidatas
        y - lista-like com o alvo
        initial_list - lista de características para começar (nomes de colunas de X)
        threshold_in - incluir uma característica se seu valor p < threshold_in
        threshold_out - excluir uma característica se seu valor p > threshold_out
        verbose - se deve imprimir a sequência de inclusões e exclusões
    Retorna: lista de características selecionadas 
    Sempre ajuste threshold_in < threshold_out para evitar looping infinito.
    Veja https://en.wikipedia.org/wiki/Stepwise_regression para detalhes.
    """
    included = list(initial_list)
    while True:
        changed = False
        # Passo avante
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print('Adicionada {:30} com valor p {:.6}'.format(best_feature, best_pval))

        # Passo retrocedente
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # Usa todos os coeficientes exceto o intercepto
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()  # Nulo se pvalues estiver vazio
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Removida {:30} com valor p {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

# Aplicando a função de seleção avante-retrocedente às variáveis independentes
variaveis = stepwise_selection(X, y)

print('Características resultantes:')
print(variaveis)

Adicionada tempo_emprego                  com valor p 0.0
#############
['tempo_emprego']
Adicionada sexo_M                         com valor p 1.01861e-159
#############
['tempo_emprego', 'sexo_M']
Adicionada tipo_renda_Assalariado         com valor p 3.16144e-64
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Assalariado']
Adicionada idade                          com valor p 1.17977e-25
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Assalariado', 'idade']
Adicionada educacao_Superior_completo     com valor p 6.36175e-15
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Assalariado', 'idade', 'educacao_Superior_completo']
Adicionada sexo_F                         com valor p 1.5132e-14
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Assalariado', 'idade', 'educacao_Superior_completo', 'sexo_F']
Adicionada posse_de_imovel                com valor p 9.38155e-09
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Assalariado', 'idade', 'educacao_Superior_complet

In [16]:
# Criando a fórmula com as variáveis selecionadas
formula = "renda ~ " + " + ".join(variaveis)

# Criando o modelo usando a fórmula e os dados de teste
modelo = smf.ols(formula, data=df_test)
reg = modelo.fit()

# Fazendo previsões no conjunto de teste
y_pred = reg.predict(X_test)

# Calculando o R²
r2 = r2_score(y_test, y_pred)
print(f"R² para o modelo final: {r2}")

R² para o modelo final: 0.23343712687284068


#### Comparando:

O R² não foi capaz de diferenciar os modelos. Portanto, apenas com essa métrica seria difícil escolher o melhor.

### Transformações

In [17]:
# Criando a fórmula com as variáveis selecionadas
formula = "renda ~ " + " + ".join(variaveis)
formula

'renda ~ tempo_emprego + sexo_M + tipo_renda_Assalariado + idade + educacao_Superior_completo + sexo_F + posse_de_imovel + tipo_renda_Servidor_público + educacao_Pós_graduação + tipo_residencia_Casa + tipo_residencia_Aluguel + tipo_renda_Pensionista'

In [18]:
# Definindo a fórmula do modelo linear
formula = 'renda ~ tempo_emprego + np.power(tempo_emprego, 2) + sexo_M + tipo_renda_Assalariado + np.log(idade) + educacao_Superior_completo + sexo_F + posse_de_imovel + tipo_renda_Servidor_público + educacao_Pós_graduação + tipo_residencia_Casa + tipo_residencia_Aluguel + tipo_renda_Pensionista'
formula

'renda ~ tempo_emprego + np.power(tempo_emprego, 2) + sexo_M + tipo_renda_Assalariado + np.log(idade) + educacao_Superior_completo + sexo_F + posse_de_imovel + tipo_renda_Servidor_público + educacao_Pós_graduação + tipo_residencia_Casa + tipo_residencia_Aluguel + tipo_renda_Pensionista'

In [19]:
# Ajustando o modelo linear com a fórmula especificada nos dados de teste
modelo = smf.ols(formula, data=df_test)
reg = modelo.fit()

# Fazendo previsões no conjunto de teste
y_pred = reg.predict(X_test)

# Avaliando o desempenho do modelo usando o coeficiente de determinação (R²)
r2_score(y_test, y_pred)

0.24919465336236968

#### Obs.: 

A transformação das variáveis 'tempo_emprego' e 'idade' permitiu que o modelo se ajustasse melhor aos dados.

### Árvore de regressão (prevendo valores contínuos)

In [20]:
from sklearn.tree import DecisionTreeRegressor

# Lista para armazenar os resultados do coeficiente de determinação (R²)
r2s = []

# Listas para armazenar os valores de i e j correspondentes aos melhores resultados
ind_i = []
ind_j = []

# Loop para testar diferentes combinações de hiperparâmetros
for i in range(2, 20):
    for j in range(20, 60):
        # Criando e treinando o modelo de árvore de decisão com os hiperparâmetros atuais
        regr_1 = DecisionTreeRegressor(max_depth=i, min_samples_leaf=j)
        regr_1.fit(X_train, y_train)
        
        # Armazenando os resultados
        ind_i.append(i)
        ind_j.append(j)
        r2s.append(regr_1.score(X_test, y_test))
        
# Criando um DataFrame com os resultados de R², profundidade e número mínimo de amostras
df_r2 = pd.DataFrame({'R2': r2s, 'profundidade': ind_i, 'n_minimo': ind_j})

# Obtendo o valor máximo de R²
max_r2 = df_r2['R2'].max()

# Imprimindo o valor máximo de R²
print(max_r2)

0.25048372626848814


#### Conclusão

O valor máximo de R² obtido com árvore de regressão é inferior ao valor máximo de R² obtido com regressão linear stepwise com transformações.